# Imports

In [1]:
import numpy as np
import queue
import tkinter as tk
from tkinter import messagebox
import random
import matplotlib.pyplot as plt
import time

# Connect Four Board class

In [2]:
class ConnectFour:
    def __init__(self, state, height, width, depth=0, parent = None, child=None):
        self.state = state
        self.width = width
        self.height = height
        self.depth= depth
        self.parent = parent
        self.child= child
    
    def isOver(self):
        for row in self.state:
            for cell in row:
                if cell == 0:
                    return False
        return True

    def dropPiece(self, col, row, player):
        self.state[col][row] = player

    def getP1Score(self):
        return self.calculateScore(1)
    
    def getP2Score(self):
        return self.calculateScore(2)
    
    def calculateScore(self, player):
        score = 0

        # Check horizontally
        for i in range(self.width):
            for j in range(self.height - 3):
                if (j + 3 < self.height and
                    self.state[i][j] == player and
                    self.state[i][j+1] == player and
                    self.state[i][j+2] == player and
                    self.state[i][j+3] == player):
                    score += 1

        # Check vertically
        for i in range(self.width - 3):
            for j in range(self.height):
                if (i + 3 < self.width and
                    self.state[i][j] == player and
                    self.state[i+1][j] == player and
                    self.state[i+2][j] == player and
                    self.state[i+3][j] == player):
                    score = score + 1

        # Check diagonally (top-left to bottom-right)
        for i in range(self.width - 3):
            for j in range(self.height - 3):
                if (i + 3 < self.width and j + 3 < self.height and
                    self.state[i][j] == player and
                    self.state[i+1][j+1] == player and
                    self.state[i+2][j+2] == player and
                    self.state[i+3][j+3] == player):
                    score += 1

        # Check diagonally (top-right to bottom-left)
        for i in range(self.width - 3):
            for j in range(3, self.height):
                if (i + 3 < self.width and j - 3 >= 0 and
                    self.state[i][j] == player and
                    self.state[i+1][j-1] == player and
                    self.state[i+2][j-2] == player and
                    self.state[i+3][j-3] == player):
                    score += 1

        return score
    
    def getWinner(self):
        p1_score = self.getP1Score()
        print(f"Player's score = {p1_score}")
        p2_score = self.getP2Score()
        print(f"AI score = {p2_score}")
        
        if p1_score > p2_score:
            return 1
        elif p2_score > p1_score:
            return 2
        else:
            return 0
        
    def evaluateGame(self):
        winner = self.getWinner()
        
        if winner == 0:
            print("It's a tie.")  
        elif winner == 1:
            print("It's player 1's victory!")
        elif winner == 2:
            print("It's player 2's victory!") 

    def is_column_full(self, col):
        return self.state[col][-1] != 0
    
    def get_player_move(self):
        while True:
            try:
                col = int(input("Enter the column where you want to drop your piece (0-6): "))
                if 0 <= col < self.width and not self.is_column_full(col):
                    return col
                else:
                    print("Invalid column! Please choose a valid column.")
            except ValueError:
                print("Invalid input! Please enter a number.") 
    
    def get_next_open_row(self, col):
        for row in range(self.height):
            if self.state[col][row] == 0:
                return row
        return -1

    def print_board(self):
        for row in range(self.height-1, -1, -1):
            for col in range(self.width):
                print(self.state[col][row], end=" ")
            print("\n")
        print("\n")

    

# Getting nodes

In [3]:
def is_valid_location(board, col):
    return board.state[board.width - 1][col] == 0

In [4]:
def find_valid_locs(board):
    emptyPos= []
    board_state = board.state

    for i in range(board.width):
        for j in range(board.height):
                if board_state[i][j] == 0:
                    emptyPos.append([i,j])
                    break
    return emptyPos


In [5]:
def get_neighbors(board, player):
    neighbors = []
    emptyPos = find_valid_locs(board)
    
    for item in emptyPos:
        new_state = [row[:] for row in board.state] 
        neighbour = ConnectFour(new_state, board.height, board.width, board.depth+1 ,board)
        neighbour.dropPiece(item[0], item[1], player)
        neighbors.append(neighbour)

    return neighbors

# Heuristic Pruning

def heuristic_Evaluation(board):
    p1_score= board.getP1Score()
    p2_score= board.getP2Score()
    potential_score = board.calculateScore(0)
    return (p2_score - p1_score) < (potential_score / 2)

In [6]:
def evaluate_window(window, player):
    score = 0
    # Switch scoring based on turn
    opponent = 2 if player == 1 else 1  # Corrected the assignment of opp_piece based on the player's value

    # Prioritize a winning move
    # Minimax makes this less important
    if window.count(player) == 4:
        score += 100
    # Make connecting 3 second priority
    elif window.count(player) == 3 and window.count(0) == 1:
        score += 5
    # Make connecting 2 third priority
    elif window.count(player) == 2 and window.count(0) == 2:
        score += 2
    # Prioritize blocking an opponent's winning move (but not over bot winning)
    if window.count(opponent) == 3 and window.count(0) == 1:
        score -= 4

    return score


In [7]:
def score_position(board, player):
    score = 0
    # Score centre column
    centre_array = [int(i) for i in list(board.state[board.width // 2])]
    centre_count = centre_array.count(player)
    score += centre_count * 3

    # Score horizontal positions
    for r in range(board.width):
        row_array = [int(i) for i in list(board.state[r])]
        for c in range(board.height - 3):
            # Create a horizontal window of 4
            window = row_array[c:c + 3]
            score += evaluate_window(window, player)

    # Score vertical positions
    for c in range(board.height):
        col_array = [int(board.state[r][c]) for r in range(board.width)]
        for r in range(board.width - 3):
            # Create a vertical window of 4
            window = col_array[r:r + 3]
            score += evaluate_window(window, player)

    # Score positive diagonals
    for r in range(board.width - 3):
        for c in range(board.height - 3):
            # Create a positive diagonal window of 4
            window = [board.state[r + i][c + i] for i in range(4)]
            score += evaluate_window(window, player)

    # Score negative diagonals
    for r in range(board.width - 3):
        for c in range(board.height - 3):
            # Create a negative diagonal window of 4
            window = [board.state[r + 3 - i][c + i] for i in range(4)]
            score += evaluate_window(window, player)

    return score


# MiniMax without Alpha-Beta Pruning

In [8]:
class MiniMax:
    def __init__(self, board, depth_limit):
        self.board = board
        self.depth_limit= depth_limit

    def construct_Tree(self):
        expanded=0
        root = self.board
        stack = [(root, 2)]
        leaf_nodes = []
        root.print_board()

        while stack:
            current_node, current_player = stack.pop()
            expanded+=1
            is_leaf = True
            #print("Parent Node:")
            #current_node.print_board()
            if current_node.depth <= self.depth_limit:
                children = get_neighbors(current_node, current_player)
                #print("Children Nodes:")
                for child in children:
                    next_player = 2 if current_player == 1 else 1
                    stack.append((child, next_player))
                    is_leaf = False
                    #child.print_board()
                #print("-----------------------------------------------------")


            if is_leaf:
                leaf_nodes.append(current_node)
        #print(f"Expaneded nodes number: {expanded}")

        return leaf_nodes
    
    def minimax(self, position, depth, maximizingPlayer):
        if maximizingPlayer:
            player = 2
        else:
            player = 1

        if depth == 0 or depth >= self.depth_limit or position.isOver():
            playerScore= score_position(position, 1)
            botScore= score_position(position, 2)
            return botScore - playerScore

        if maximizingPlayer:
            maxEval = -float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, False)
                maxEval = max(maxEval, eval)
            return maxEval
        else:
            minEval = float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, True)  # Corrected to pass True for maximizing player
                minEval = min(minEval, eval)
            return minEval
            

# MiniMax with Alpha-Beta Pruning

### Working

In [9]:
class MiniMaxAlphaBeta:
    def __init__(self, board, depth_limit):
        self.board = board
        self.depth_limit= depth_limit

    def construct_Tree(self):
        root = self.board
        stack = [(root, 2)]
        leaf_nodes = []
        expanded=0
        root.print_board()


        while stack:
            current_node, current_player = stack.pop()
            print("Parent Node:")
            current_node.print_board()
            expanded+=1

            is_leaf = True
            if current_node.depth <= self.depth_limit:
                children = get_neighbors(current_node, current_player)
                print("Children Nodes:")
                for child in children:
                    next_player = 2 if current_player == 1 else 1
                    stack.append((child, next_player))
                    is_leaf = False
                    child.print_board()
                print("-----------------------------------------------------")

            if is_leaf:
                leaf_nodes.append(current_node)
                
        print(f"Expaneded nodes number: {expanded}")
        return leaf_nodes
    
    def minimaxAB(self, position, depth, maximizingPlayer, alpha= -float('inf'), beta= float('inf')):
        if maximizingPlayer:
            player = 2
        else:
            player = 1

        if depth == 0 or depth >= self.depth_limit or position.isOver():
            playerScore= score_position(position, 1)
            botScore= score_position(position, 2)
            return botScore - playerScore

        if maximizingPlayer:
            maxEval = -float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, False, alpha, beta)
                maxEval = max(maxEval, eval)
                alpha= max(alpha, eval)
                if beta <= alpha:
                    break
            return maxEval
        else:
            minEval = float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, True, alpha, beta)  # Corrected to pass True for maximizing player
                minEval = min(minEval, eval)
                beta= min(beta, eval)
                if beta <= alpha:
                    break
            return minEval
            

### Reference

class AlphaBetaPruning:
    def __init__(self, board, depth_limit):
            self.board = board
            self.depth_limit = depth_limit

    def construct_Tree(self):
        root = self.board
        stack = [(root, 2)]
        leaf_nodes = []

        while stack:
            current_node, current_player = stack.pop()
            is_leaf = True
            if current_node.depth <= self.depth_limit:
                children = get_neighbors(current_node, current_player)
                for child in children:
                    next_player = 2 if current_player == 1 else 1
                    stack.append((child, next_player))
                    is_leaf = False

            if is_leaf:
                leaf_nodes.append(current_node)

        return leaf_nodes
    
    def alpha_beta(self, position, depth, alpha, beta, maximizingPlayer):
        if maximizingPlayer:
            player = 2
        else:
            player = 1

        if depth == 0 or depth >= self.depth_limit or position.isOver():
            playerScore = score_position(position, 1)
            botScore = score_position(position, 2)
            return botScore - playerScore

        if maximizingPlayer:
            maxEval = -float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, alpha, beta, False)
                maxEval = max(maxEval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cut-off
            return maxEval
        else:
            minEval = float('inf')
            for child in position:  # Assuming position returns a list of possible child positions
                eval = self.minimax(child, depth - 1, alpha, beta, True)
                minEval = min(minEval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cut-off
            return minEval

# ExpectiMinMax

### REF

class ExpectiMiniMax:
    def __init__(self, board, depth_limit):
        self.board = board
        self.depth_limit = depth_limit

    def construct_Tree(self):
        expanded = 0
        root = self.board
        stack = [(root, 2)]
        leaf_nodes = []

        while stack:
            current_node, current_player = stack.pop()
            expanded += 1

            is_leaf = True
            if current_node.depth <= self.depth_limit:
                children, probabilities = self.get_child_nodes_with_probabilities(current_node, current_player)
                for child, prob in zip(children, probabilities):
                    next_player = 2 if current_player == 1 else 1
                    stack.append((child, next_player))
                    is_leaf = False

            if is_leaf:
                leaf_nodes.append(current_node)

        return leaf_nodes

    def get_child_nodes_with_probabilities(self, board, player):
        children = []
        probabilities = []
        empty_positions = find_valid_locs(board)

        # Probability distribution for the chosen column
        chosen_prob = 0.6
        side_prob = (1 - chosen_prob) / 2

        for pos in empty_positions:
            col, row = pos
            new_state = [row[:] for row in board.state]
            new_board = ConnectFour(new_state, board.height, board.width, board.depth + 1)
            new_board.dropPiece(col, row, player)
            children.append(new_board)

            # Assign probabilities
            if col == empty_positions[0][0]:  # Leftmost column
                probabilities.append((chosen_prob, side_prob))
            elif col == empty_positions[-1][0]:  # Rightmost column
                probabilities.append((side_prob, chosen_prob))
            else:
                probabilities.append((side_prob, chosen_prob, side_prob))

        return children, probabilities

    def expectiminimax(self, position, depth, maximizing_player):
        if maximizing_player:
            player = 2
        else:
            player = 1

        if depth == 0 or position.isOver():
            player_score = score_position(position, 1)
            bot_score = score_position(position, 2)
            return bot_score - player_score

        if maximizing_player:  # Bot's turn
            max_value = float('-inf')
            for child, prob in zip(*self.get_child_nodes_with_probabilities(position, player)):
                # Evaluate the child node recursively
                child_value = self.expectiminimax(child, depth - 1, False)

                # Update the maximum value considering the probability of the child node
                max_value = max(max_value, prob[0] * child_value)

            return max_value
        else:  # Player's turn
            min_value = float('inf')
            for child, prob in zip(*self.get_child_nodes_with_probabilities(position, player)):
                # Evaluate the child node recursively
                child_value = self.expectiminimax(child, depth - 1, True)

                # Update the minimum value considering the probability of the child node
                min_value = min(min_value, prob[0] * child_value)

            return min_value


### WORK

In [10]:
import random

class ExpectiMiniMax:
    def __init__(self, board, depth_limit):
        self.board = board
        self.depth_limit = depth_limit

    def construct_Tree(self):
        expanded=0
        root = self.board
        stack = [(root, 2)]
        leaf_nodes = []
        print("Parent Node:")
        root.print_board()

        while stack:
            current_node, current_player = stack.pop()
            expanded +=1
            current_node.print_board()

            is_leaf = True
            if current_node.depth <= self.depth_limit:
                children, probabilities = self.get_child_nodes_with_probabilities(current_node, current_player)
                print("Children Nodes:")
                for child, prob in zip(children, probabilities):
                    next_player = 2 if current_player == 1 else 1
                    stack.append((child, next_player))
                    is_leaf = False
                print("-----------------------------------------------------")
            if is_leaf:
                leaf_nodes.append(current_node)

        return leaf_nodes

    def get_child_nodes_with_probabilities(self, board, player):
        children = []
        probabilities = []
        emptyPos = find_valid_locs(board)

        # Probability distribution for the chosen column
        chosen_prob = 0.6
        side_prob = (1 - chosen_prob) / 2

        for item in emptyPos:
            new_state = [row[:] for row in board.state] 
            child = ConnectFour(new_state, board.height, board.width, board.depth+1 ,board)
            child.dropPiece(item[0], item[1], player)
            children.append(child)

            # Assign probabilities
            if item[0] == emptyPos[0][0]:  # Leftmost column
                probabilities.append((chosen_prob, side_prob, side_prob))
            elif item[0] == emptyPos[-1][0]:  # Rightmost column
                probabilities.append((side_prob, side_prob, chosen_prob))
            else:
                probabilities.append((side_prob, chosen_prob, side_prob))

        return children, probabilities


    def expectiminimax(self, position, depth, maximizingPlayer):
        if maximizingPlayer:
            player = 2
        else:
            player = 1

        if depth == 0 or depth >= self.depth_limit or position.isOver():
            player_score = score_position(position, 1)
            bot_score = score_position(position, 2)
            return bot_score - player_score

        if maximizingPlayer:  # Bot's turn
            max_value = -float('inf')
            for child, prob in zip(self.get_child_nodes_with_probabilities(position, player)):
                # Evaluate the child node recursively
                child_value = self.expectiminimax(child, depth - 1, False)

                # Update the maximum value considering the probability of the child node
                max_value += prob[0] * child_value

            return max_value
        else:  # Player's turn
            min_value = float('inf')
            for child, prob in zip(self.get_child_nodes_with_probabilities(position, player)):
                # Evaluate the child node recursively
                child_value = self.expectiminimax(child, depth - 1, True)

                # Update the minimum value considering the probability of the child node
                min_value += prob[0] * child_value

            return min_value



# GUI

### REF

def play_connect_four(height, width, depth_limit, flag):
    # Initialize the game
    initialState = [[0] * height for _ in range(width)]
    cf = ConnectFour(initialState, height, width)
    mm = MiniMax(cf, depth_limit)
    ab = AlphaBetaPruning(cf, depth_limit)

    # Print the initial board
    cf.print_board()

    if flag == 1:
        # Game loop using not alpha beta
        while True:
            # Player's turn
            player_move = cf.get_player_move()
            row = cf.get_next_open_row(player_move)
            if row == -1:
                print("Column is full! Please choose another column.")
                continue
            cf.dropPiece(player_move, row, 1)
            print(cf.state)

            # MiniMax agent's turn
            leaf_nodes = mm.construct_Tree()
            max_score = -float('inf')
            best_move = None
            for node in leaf_nodes:
                score = mm.minimax(node, depth_limit, False)
                if score > max_score:
                    max_score = score
                    best_move = node
            for i in range(depth_limit):
                if best_move.parent != None:
                    best_move = best_move.parent
            cf.state = best_move.state  # Update the board with the agent's move
            cf.print_board()
            if cf.isOver():
                break

    elif flag == 2:
        # Game loop using alpha beta
        while True:
            # Player's turn
            player_move = cf.get_player_move()
            row = cf.get_next_open_row(player_move)
            if row == -1:
                print("Column is full! Please choose another column.")
                continue
            cf.dropPiece(player_move, row, 1)
            print(cf.state)

            # MiniMax agent's turn
            leaf_nodes = ab.construct_Tree()
            max_score = -float('inf')
            best_move = None
            for node in leaf_nodes:
                score = ab.alpha_beta(node, depth_limit)
                if score > max_score:
                    max_score = score
                    best_move = node
            for i in range(depth_limit):
                if best_move.parent != None:
                    best_move = best_move.parent
            cf.state = best_move.state  # Update the board with the agent's move
            cf.print_board()
            if cf.isOver():
                break


    # Check if the game is over
    if cf.isOver():

        # Check if the player wins
        if cf.getWinner() == 1:
            print("Congratulations! You win!")
        
        # Check if the agent wins
        elif cf.getWinner() == 2:
            print("The MiniMax agent wins!")
        else:
            print("The game is a tie!")

# Game Functions

In [11]:
class ConnectFourGUI:
    def __init__(self, root, height, width, depth_limit, f):
        self.root = root
        self.height = height
        self.width = width
        self.depth_limit = depth_limit
        self.f = f

        self.root.title("Connect Four")
        self.canvas = tk.Canvas(root, width=width*50, height=height*50, bg='#1976D2')
        self.canvas.pack()

        self.initialize_game()

    def initialize_game(self):
        self.cf = ConnectFour([[0] * self.height for _ in range(self.width)], self.height, self.width)
        self.draw_board()
        self.current_player = 1
        self.game_over = False
        self.player_turn = True

    def draw_board(self):
        self.canvas.delete("all")
        for row in range(self.height):
            for col in range(self.width):
                x1 = col * 50
                y1 = (self.height - row - 1) * 50  # Flip the y-coordinate
                x2 = x1 + 50
                y2 = y1 + 50
                self.canvas.create_rectangle(x1, y1, x2, y2, fill='#1D63F2', outline='#FFFFFF')
                if self.cf.state[col][row] == 1:  # Adjust row indexing
                    self.canvas.create_oval(x1 + 5, y1 + 5, x2 - 5, y2 - 5, fill='#FBC910')
                elif self.cf.state[col][row] == 2:  # Adjust row indexing
                    self.canvas.create_oval(x1 + 5, y1 + 5, x2 - 5, y2 - 5, fill='#CB1B2D')
                else:
                    self.canvas.create_oval(x1 + 5, y1 + 5, x2 - 5, y2 - 5, fill='#FFFFFF')

    def drop_piece(self, column):
        if not self.game_over and self.player_turn:
            row = self.cf.get_next_open_row(column)
            if row != -1:
                self.cf.dropPiece(column, row, self.current_player)
                self.draw_board()
                if self.cf.isOver():
                    self.game_over = True
                    if self.cf.getWinner() == 1:
                        messagebox.showinfo("Game Over", "Congratulations! You win!")
                    elif self.cf.getWinner() == 2:
                        messagebox.showinfo("Game Over", "The MiniMax agent wins!")
                    else:
                        messagebox.showinfo("Game Over", "The game is a tie!")
                else:
                    self.player_turn = False
                    if self.f == 1:
                        self.root.after(1000, self.ai_move_minimax)  # Call AI move after a delay
                    elif self.f == 2:
                        self.root.after(1000, self.ai_move_alpha_beta)  # Call AI move after a delay
                    elif self.f == 3:
                        self.root.after(1000, self.ai_move_expecti)

        

    def ai_move_minimax(self):
        start= time.time()
        if not self.game_over and not self.player_turn:
            mm = MiniMax(self.cf, self.depth_limit)
            leaf_nodes = mm.construct_Tree()
            max_score = -float('inf')
            best_move = None
            for node in leaf_nodes:
                score = mm.minimax(node, self.depth_limit, False)
                if score > max_score:
                    max_score = score
                    best_move = node
            for i in range(self.depth_limit):
                if best_move.parent != None:
                    best_move = best_move.parent
            self.cf.state = best_move.state
            self.draw_board()
            end= time.time()
            taken= end-start
            print(f"Time taken for this move: {taken}")
            if self.cf.isOver():
                self.game_over = True
                if self.cf.getWinner() == 1:
                    messagebox.showinfo("Game Over", "Congratulations! You win!")
                elif self.cf.getWinner() == 2:
                    messagebox.showinfo("Game Over", "The MiniMax agent wins!")
                else:
                    messagebox.showinfo("Game Over", "The game is a tie!")
            else:
                self.player_turn = True


    def ai_move_alpha_beta(self):
        if not self.game_over and not self.player_turn:
            start= time.time()
            mm = MiniMaxAlphaBeta(self.cf, self.depth_limit)
            leaf_nodes= mm.construct_Tree()
            max_score = -float('inf')
            best_move = None
            for node in leaf_nodes:
                score = mm.minimaxAB(node, self.depth_limit, False)
                if score > max_score:
                    max_score = score
                    best_move = node
            for i in range(self.depth_limit):
                if best_move.parent != None:
                    best_move = best_move.parent
            self.cf.state = best_move.state
            self.draw_board()
            end= time.time()
            taken= end-start
            print(f"Time taken for this move: {taken}")
            if self.cf.isOver():
                self.game_over = True
                if self.cf.getWinner() == 1:
                    messagebox.showinfo("Game Over", "Congratulations! You win!")
                elif self.cf.getWinner() == 2:
                    messagebox.showinfo("Game Over", "The MiniMax agent wins!")
                else:
                    messagebox.showinfo("Game Over", "The game is a tie!")
            else:
                self.player_turn = True
    
    def ai_move_expecti(self):
        if not self.game_over and not self.player_turn:
            start= time.time()
            mm = ExpectiMiniMax(self.cf, self.depth_limit)
            leaf_nodes= mm.construct_Tree()
            max_score = -float('inf')
            best_move = None
            for node in leaf_nodes:
                score = mm.expectiminimax(node, self.depth_limit, False)
                if score > max_score:
                    max_score = score
                    best_move = node
            for i in range(self.depth_limit):
                if best_move.parent != None:
                    best_move = best_move.parent
            self.cf.state = best_move.state
            self.draw_board()
            end= time.time()
            taken= end-start
            print(f"Time taken for this move: {taken}")
            if self.cf.isOver():
                self.game_over = True
                if self.cf.getWinner() == 1:
                    messagebox.showinfo("Game Over", "Congratulations! You win!")
                elif self.cf.getWinner() == 2:
                    messagebox.showinfo("Game Over", "The MiniMax agent wins!")
                else:
                    messagebox.showinfo("Game Over", "The game is a tie!")
            else:
                self.player_turn = True


# Usage

In [12]:
root = tk.Tk()
height = 6
width = 7
depth_limit = 6


def init_game(f):
    game = ConnectFourGUI(root, height, width, depth_limit, f)
    buttons = []
    for col in range(width):
        button = tk.Button(root, text=str(col+1), command=lambda col=col: game.drop_piece(col), bg='#607D8B', fg='#FFFFFF')
        button.pack(side=tk.LEFT)
        buttons.append(button)
    
    

button_1 = tk.Button(root, text="MiniMax", command=lambda: init_game(1))
button_1.pack(side=tk.LEFT)
button_2 = tk.Button(root, text="AlphaBeta", command=lambda: init_game(2))
button_2.pack(side=tk.LEFT)
button_3 = tk.Button(root, text="ExpectiMinMax", command=lambda: init_game(3))
button_3.pack(side=tk.LEFT)
root.mainloop()


0 0 0 0 0 0 0 

0 0 0 0 0 0 0 

0 0 0 0 0 0 0 

0 0 0 0 0 0 0 

0 0 0 0 0 0 0 

0 0 0 1 0 0 0 





: 